# This directory is made according to colab

In [ ]:
import torch

# Check PyTorch version
print("PyTorch version:", torch.__version__)

# Check CUDA version
print("CUDA version:", torch.version.cuda)

# Check if CUDA is available
print("CUDA is available:", torch.cuda.is_available())

In [ ]:
!pip install bitsandbytes # requires 0.44.1 version

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 transformers==4.31.0 trl==0.4.7 tensorboard ngrok

Import Necessary Modules

In [ ]:
import pandas as pd
from datasets import Dataset
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

This data is already in the required format for training

In [ ]:
# Load dataset (you can process it here)
# dataset = load_dataset(dataset_name, split="train")
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/neer_product_format_data.csv')
df.head(2)

X = df.drop(['formatted_text'], axis = 1)
y = df['formatted_text']

y

Train test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)


y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [ ]:
# Convert the DataFrame to Hugging Face Dataset
dataset1 = Dataset.from_pandas(y_train)
eval_dataset1 = Dataset.from_pandas(y_test)

# Inspect the Hugging Face dataset
print(dataset1)

In [ ]:
# Hugging face login
from huggingface_hub import login


HF_TOKEN = "Give your HuggingFace token here"
# Replace 'your_hf_token' with the token you copied from your Hugging Face account
login(token=HF_TOKEN)

Required Configuration 

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
# dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "finetuned_llama"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.25

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "results-llama"

# Number of training epochs
num_train_epochs = 3

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 2

# Batch size per GPU for evaluation
per_device_eval_batch_size = 2

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 2

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# metric
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Set up 4-bit quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16 (optional)
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1  # This config may vary depending on your setup

n = 3  # Reduce the number of transformer blocks to fine-tune

for name, param in model.named_parameters():
    if param.dtype in [torch.float16, torch.float32, torch.bfloat16, torch.float64]:
        layer_index = int(name.split('.')[2]) if "transformer.h." in name else None
        if layer_index is not None and layer_index >= model.config.num_hidden_layers - n:
            param.requires_grad = True
        else:
            param.requires_grad = False

# # Enable mixed precision and gradient checkpointing
model.gradient_checkpointing_enable()  # Try enabling this again


# Loading tokeniser and model form HuggingFace

In [ ]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # Ensures compatibility with padding
tokenizer.padding_side = "right"  # Prevents issues with fp16 training

# Load LoRA configuration for QLoRA
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",  # Adaptation for causal language modeling
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,  # Enable mixed precision if supported
    bf16=bf16,  # Enable bf16 if supported
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",  # Logs data to Tensorboard for easy visualization

    # Save and evaluation strategy
    evaluation_strategy="steps",  # Also can be "epoch"
    save_strategy="steps",  # Save every few steps
    save_total_limit=3,  # Keep the last 3 checkpoints
    load_best_model_at_end=True,  # Load the best model after training
    metric_for_best_model="eval_loss",  # Use validation loss as the metric for best model
    greater_is_better=False,  # Lower validation loss is better
)

# Set supervised fine-tuning trainer with early stopping
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset1,
    eval_dataset=eval_dataset1,  # Your evaluation dataset
    peft_config=peft_config,
    dataset_text_field="formatted_text",  # Field containing the text in your dataset
    max_seq_length=max_seq_length,  # Maximum sequence length for the inputs
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,  # Enable or disable sequence packing
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.01)],  # Early stopping added here
    # compute_metrics=compute_metrics,  # compute
)

In [ ]:
# Train model
trainer.train()

# Save the model for future usage

In [ ]:
# saving the configurration.json into the output directory
# Save the model configuration
model.config.save_pretrained(output_dir)
# Explicitly save the model, tokenizer, and configuration
trainer.save_model(output_dir)  # Saves the model and configuration to `output_dir`
tokenizer.save_pretrained(output_dir)  # Saves the tokenizer